---

<h2 style="text-align: center;"><b>Свёрточные нейронные сети: CIFAR10</b></h3>

Выполнила: Трофимова Екатерина Александровна, 20223

---

### Теория CNN

В этом ноутбке мы посмотрим, насколько хорошо CNN будут предсказывать классы на более сложном датасете картинок -- CIFAR10. 

**Внимание:** Рассматривается ***задача классификации изображений***.

***Свёрточная нейросеть (Convolutional Neural Network, CNN)*** - это многослойная нейросеть, имеющая в своей архитектуре помимо *полносвязных слоёв* (а иногда их может и не быть) ещё и **свёрточные слои (Conv Layers)** и **pooling-слои (Pool Layers)**.  

Собственно, название такое эти сети получили потому, что в основе их работы лежит операция **свёртки**.

Сразу же стоит сказать, что свёрточные нейросети **были придуманы прежде всего для задач, связанных с изображениями**, следовательно, на вход они тоже "ожидают" изображение.

* Например, вот так выглядит неглубокая свёрточная нейросеть, имеющая такую архитектуру:  
`Input -> Conv 5x5 -> Pool 2x2 -> Conv 5x5 -> Pool 2x2 -> FC -> Output`

<img src="https://camo.githubusercontent.com/269e3903f62eb2c4d13ac4c9ab979510010f8968/68747470733a2f2f7261772e6769746875622e636f6d2f746176677265656e2f6c616e647573655f636c617373696669636174696f6e2f6d61737465722f66696c652f636e6e2e706e673f7261773d74727565" width=800, height=600>  
  
Свёрточные нейросети (простые, есть и намного более продвинутые) почти всегда строятся по следующему правилу:  

`INPUT -> [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]*L -> FC`  

то есть:  

1). ***Входной слой***: batch картинок -- тензор размера `(batch_size, H, W, C)` или `(batch_size, C, H, W)`

2). $M$ блоков (M $\ge$ 0) из свёрток и pooling-ов, причём именно в том порядке, как в формуле выше. Все эти $M$ блоков вместе называют ***feature extractor*** свёрточной нейросети, потому что эта часть сети отвечает непосредственно за формирование новых, более сложных признаков поверх тех, которые подаются (то есть, по аналогии с MLP, мы опять же переходим к новому признаковому пространству, однако здесь оно строится сложнее, чем в обычных многослойных сетях, поскольку используется операция свёртки)  

3). $L$ штук FullyConnected-слоёв (с активациями). Эту часть из $L$ FC-слоёв называют ***classificator***, поскольку эти слои отвечают непосредственно за предсказание нужно класса (сейчас рассматривается задача классификации изображений).


<h3 style="text-align: center;"><b>Свёрточная нейросеть на PyTorch</b></h3>

Ешё раз напомним про основные компоненты нейросети:

- непосредственно, сама **архитектура** нейросети (сюда входят типы функций активации у каждого нейрона);
- начальная **инициализация** весов каждого слоя;
- метод **оптимизации** нейросети (сюда ещё входит метод изменения `learning_rate`);
- размер **батчей** (`batch_size`);
- количетсво **эпох** обучения (`num_epochs`);
- **функция потерь** (`loss`);  
- тип **регуляризации** нейросети (`weight_decay`, для каждого слоя можно свой);  

То, что связано с ***данными и задачей***:  
- само **качество** выборки (непротиворечивость, чистота, корректность постановки задачи);  
- **размер** выборки;  

Так как мы сейчас рассматриваем **архитектуру CNN**, то, помимо этих компонент, в свёрточной нейросети можно настроить следующие вещи:  

- (в каждом ConvLayer) **размер фильтров (окна свёртки)** (`kernel_size`)
- (в каждом ConvLayer) **количество фильтров** (`out_channels`)  
- (в каждом ConvLayer) размер **шага окна свёртки (stride)** (`stride`)  
- (в каждом ConvLayer) **тип padding'а** (`padding`)  


- (в каждом PoolLayer) **размер окна pooling'a** (`kernel_size`)  
- (в каждом PoolLayer) **шаг окна pooling'а** (`stride`)  
- (в каждом PoolLayer) **тип pooling'а** (`pool_type`)  
- (в каждом PoolLayer) **тип padding'а** (`padding`)

### CIFAR10

<img src="https://raw.githubusercontent.com/soumith/ex/gh-pages/assets/cifar10.png" width=500, height=400>

**CIFAR10:** это набор из 60k картинок 32х32х3, 50k которых составляют обучающую выборку, и оставшиеся 10k - тестовую. Классов в этом датасете 10: `'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'`.

In [ ]:
# !pip install torch torchvision

In [ ]:
import torch
import torchvision
from torchvision import transforms
from tqdm import tqdm_notebook

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='../pytorch_data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../pytorch_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
trainset.data

In [ ]:
trainloader.dataset.train_list[0]

In [ ]:
# случайный индекс от 0 до размера тренировочной выборки
i = np.random.randint(low=0, high=50000)

plt.imshow(trainloader.dataset.data[i]);

### CNN для предсказания на CIFAR10.

Напишем свёрточную нейросеть для предсказания на CIFAR10

In [ ]:
# Подключение зависимостей

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
from torch.optim import lr_scheduler

### Вспомогательные функции

In [ ]:
# Попытка ускорить вычисления за счет gpu

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
#trainloader = DeviceDataLoader(trainloader, device)
#testloader = DeviceDataLoader(testloader, device)

In [ ]:
# Функция для обучения модели

def train(net, epoch_num = 5, learning_rate = 1e-3):

  loss_fn = torch.nn.CrossEntropyLoss()

  optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
  # динамически изменяем LR
  scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=epoch_num)

  # итерируемся
  for epoch in tqdm_notebook(range(epoch_num)):
    
    scheduler.step()

    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(trainloader)):
        # так получаем текущий батч
        X_batch, y_batch = batch

        # обнуляем веса
        optimizer.zero_grad()

        # forward + backward + optimize
        y_pred = net(X_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # выводим качество каждые 2000 батчей
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

  print('Обучение закончено')

In [ ]:
# Функция для проверки качества

def check_accuracy(net):
  class_correct = list(0. for i in range(10))
  class_total = list(0. for i in range(10))

  with torch.no_grad():
    for data in testloader:
        images, labels = data
        y_pred = net(images)
        _, predicted = torch.max(y_pred, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

  avg_accuracy = 0

  for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
    avg_accuracy += 100 * class_correct[i] / class_total[i]

  print('Avg accuracy %2d %%' % (avg_accuracy / 10))

In [ ]:
# Функция для визуальной проверки результата

def visualize_result(net, index):
    image = testloader.dataset.data[index]
    plt.imshow(image)
    
    image = transform(image)  # не забудем отмасштабировать!
    
    y_pred = net(image.view(1, 3, 32, 32))
    _, predicted = torch.max(y_pred, 1)
    
    plt.title(f'Predicted: {classes[predicted.numpy()[0]]}')

###Базовая архитектура

In [ ]:
class SimpleConvNet(torch.nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(SimpleConvNet, self).__init__()
        # feature extractor
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # classificator
        self.fc1 = nn.Linear(5 * 5 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 5 * 5 * 16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = SimpleConvNet()
train(net, learning_rate=0.001, epoch_num=10)

Посмотрим на accuracy на тестовом датасете:

In [ ]:
check_accuracy(net)

При базовой архитектуре наблюдается средняя точность в районе 64% на 10 эпохах. Минимальная точность класса 44%. Среднее время вычисления на эпоху = 1:10

Проверим работу нейросети визуально (позапускайте ячейку несколько раз):

In [ ]:
i = np.random.randint(low=0, high=10000)
visualize_result(net, i)

###Эксперименты с числом сверточных слоев и каналов



Попробуем просто добавить новый сверточный слой 

In [ ]:
class ConvNet_3CL(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3CL, self).__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        
        self.fc1 = nn.Linear(3 * 3 * 32, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 3 * 3 * 32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3CL()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Добавление еще одного сверточного слоя с малым количеством каналов отрицательно сказалось на качестве обучения (средний результат ухудшился до 61%) и времени обучения. Примерно на 9 эпохе процесс обучения застопорился. Попробуем теперь вернуться к 2м сверточным слоям, но увеличим число каналов

In [ ]:
class ConvNet_2Cl(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_2Cl, self).__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        
        self.fc1 = nn.Linear(5 * 5 * 128, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 5 * 5 * 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_2Cl()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Увеличение числа каналов положительно сказалось на средней точности - 71% против базовых 64%. Но обучение в рамках эпохи теперь идет гораздо дольше. Попробуем одновременно увеличить число сверточных слоев и число каналов

In [ ]:
class ConvNet_3CL_CH(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3CL_CH, self).__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5)
        
        self.fc1 = nn.Linear(256, 120) # 1 x 1 x 256
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3CL_CH()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Средняя точность стала чуть ниже - 70% против 71% на двух слоях. При этом сильно возросло время обучения. Теперь попробуем изменить размер ядра свертки для случая с двумя слоями

In [ ]:
class ConvNet_2Cl_3KS(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_2Cl_3KS, self).__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        
        self.fc1 = nn.Linear(6 * 6 * 128, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 6 * 6 * 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_2Cl_3KS()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Для двух слоев изменение размера ядра свертки не дало существенных изменений. Теперь посмотрим на 3х слоях

In [ ]:
class ConvNet_3CL_CH_3KS(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3CL_CH_3KS, self).__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        
        self.fc1 = nn.Linear(2 * 2 * 256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 2 * 2 * 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3CL_CH_3KS()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

А вот на 3х слоях уже наблюдается небольшой прирост: 73% против 70%. Минимальная точность возросладо 54%. 

Добавление слоев и каналов позволило обогатить пространство признаков и улучшить тем самым результат классификации

### Эксперименты с пулингом и нормализацией

Теперь попробуем поменять тип пулинга с max на avg

In [ ]:
class ConvNet_3Cl_3KS_AvgPool(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3Cl_3KS_AvgPool, self).__init__()
        
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        
        self.fc1 = nn.Linear(2 * 2 * 256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 2 * 2 * 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3Cl_3KS_AvgPool()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Смена типа пулинга с max на avg еще немного улучшила результат: средняя точность 75% против 73%, минимальная - 57% против 54%. Т.е. положение признака оказалось немного важнее его нличия. Попробуем добавить нормализацию

In [ ]:
class ConvNet_3Cl_3KS_AvgPool_BN(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3Cl_3KS_AvgPool_BN, self).__init__()
        
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.fc1 = nn.Linear(2 * 2 * 256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.pool(x)
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 2 * 2 * 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3Cl_3KS_AvgPool_BN()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Нормализация не повлияла на результат

### Эксперимент с функцией активации

Обычно в качестве функции активации сверточных слоев используют функцию ReLU. Рассматривать функции активации вроде сигмоидной или тангенциальной мы не будем, т.к. они приводят к проблемам с затуханием или увеличением градиентов. Вместо этого попробуем использовать ELU, которая сохраняет преимущества ReLU и помогает избежать проблемы умирающего ReLU

In [ ]:
class ConvNet_3Cl_3KS_AvgPool_ELU(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3Cl_3KS_AvgPool_ELU, self).__init__()
        
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        
        self.fc1 = nn.Linear(2 * 2 * 256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = self.pool(x)
        x = F.elu(self.conv2(x))
        x = self.pool(x)
        x = F.elu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 2 * 2 * 256)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3Cl_3KS_AvgPool_ELU()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Несмотря на то, что значение функции потерь теперь меньше, точность тоже упала. ELU не дает нам выигрыша на текущей архитектуре

### Эксперимент с числом полносвязных слоев

Начнем с простого - уберем 1 слой

In [ ]:
class ConvNet_3Cl_3KS_AvgPool_2Fl(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3Cl_3KS_AvgPool_2Fl, self).__init__()
        
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        
        self.fc1 = nn.Linear(2 * 2 * 256, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = self.pool(x)
        x = F.elu(self.conv2(x))
        x = self.pool(x)
        x = F.elu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 2 * 2 * 256)
        x = F.elu(self.fc1(x))
        x = self.fc3(x)
        return x

In [ ]:
net = ConvNet_3Cl_3KS_AvgPool_2Fl()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

И наоборот - добавим 1 слой

In [ ]:
class ConvNet_3Cl_3KS_AvgPool_4Fl(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(ConvNet_3Cl_3KS_AvgPool_4Fl, self).__init__()
        
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        
        self.fc1 = nn.Linear(2 * 2 * 256, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = self.pool(x)
        x = F.elu(self.conv2(x))
        x = self.pool(x)
        x = F.elu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 2 * 2 * 256)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
net = ConvNet_3Cl_3KS_AvgPool_4Fl()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

В обоих случаях точность классификации снизилась. Для текущей архитектуры оптимальным является наличие 3 линейных слоев

### Сильная архитектура, которая уже была (!) в ноутбуке

Попробуем обучить ещё более сильную нейросеть:

In [ ]:
class StrongConvNet(nn.Module):
    def __init__(self):
        # вызов конструктора класса nn.Module()
        super(StrongConvNet, self).__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.dropout = nn.Dropout(p=0.2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.bn5 = nn.BatchNorm2d(32)
        
        self.fc1 = nn.Linear(4 * 4 * 32, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.pool(x)
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.pool(x)
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.bn5(F.relu(self.conv5(x)))
#         print(x.shape)
        x = x.view(-1, 4 * 4 * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Обучим:

In [ ]:
net = StrongConvNet()
train(net, learning_rate=0.001, epoch_num=10)

In [ ]:
check_accuracy(net)

Посмотрим визуально на работу нейросети:

In [ ]:
i = np.random.randint(low=0, high=10000)
visualize_result(i)

### Лучшая архитектура

Итого: наилучший результат 75% точности в среднем, максимальной точности в 85% для отдельного класса и минимальной точности в 57% для отдельного класса. Результат был достигнут при 3х сверточных слоях с увеличенным числом каналов и меньшим ядром свертки для получения большего пространства признаков (что приводит также к сильному увеличению времени обучения), с avg пулингом, ReLU в качестве функции активации и 3мя линейными полносвязными слоями в качестве классификатора. 

Даже обучив более глубокую и прокаченную (BatchNorm, Dropout) нейросеть на этих данных мы видим, что качество нас всё ещё не устраивает, в реальной жизни необходимо ошибаться не больше, чем на 5%, а часто и это уже много. Как же быть, ведь свёрточные нейросети должны хорошо классифицировать изображения?  

К сожалению, обучение нейросети с нуля на не очень большой выборке (а здесь она именно такая) часто приводит к переобучению, что плохо сказывается на тестовом качестве.  

Для того, чтобы получить более качественную модель, часто **до**обучают сильную нейросеть, обученную на ImageNet, то есть используют технику Transfer Learning. О ней речь пойдёт далее в нашем курсе.

<h3 style="text-align: center;"><b>Полезные ссылки</b></h3>

1). *Примеры написания нейросетей на PyTorch (официальные туториалы) (на английском): https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#examples  
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html*

2). Курс Стэнфорда:  http://cs231n.github.io/

3). Практически исчерпывающая информация по основам свёрточных нейросетей (из cs231n) (на английском):  

http://cs231n.github.io/convolutional-networks/  
http://cs231n.github.io/understanding-cnn/  
http://cs231n.github.io/transfer-learning/

4). Видео о Computer Vision от Andrej Karpathy: https://www.youtube.com/watch?v=u6aEYuemt0M